# load necessary stuffs

In [1]:
# download link: https://github.com/obophenotype/human-phenotype-ontology/releases
# phenotype.hpoa : https://github.com/obophenotype/human-phenotype-ontology/blob/fd050e5aea1d01bd3e6b4e524acb7289e1f3e266/docs/annotations/genes_to_phenotype.md
# phenotype_to_genes.txt : https://github.com/obophenotype/human-phenotype-ontology/blob/fd050e5aea1d01bd3e6b4e524acb7289e1f3e266/docs/annotations/phenotype_to_genes.md
# genes_to_phenotype.txt : https://github.com/obophenotype/human-phenotype-ontology/blob/fd050e5aea1d01bd3e6b4e524acb7289e1f3e266/docs/annotations/genes_to_phenotype.md

import obonet
import polars as pl
import numpy as np
import torch

class Configs:
    hpo_files_path = "docs/"
    snomed_file_path = "snomed/"
    embedding_model_hf = "cambridgeltl/SapBERT-from-PubMedBERT-fulltext"
    output_embedding_folder = "embeddings_sapbert_and_info/"
    output_mapping_folder = "output_mapping/"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
def read_txt(file_name):
    return pl.read_csv(file_name, separator="\t", comment_prefix = "#")
def read_parquet(file_name, cols):
    return pl.read_parquet(file_name, columns=cols)

In [3]:
# Load ontology
graph = obonet.read_obo(f"{Configs.hpo_files_path}hp.obo")

# Load phenotype-disease mappings
df_hpoa = read_txt(f"{Configs.hpo_files_path}phenotype.hpoa")
df_p2g = read_txt(f"{Configs.hpo_files_path}phenotype_to_genes.txt")
df_g2p = read_txt(f"{Configs.hpo_files_path}genes_to_phenotype.txt")

# get release data (for label and synonym)
df_snomed_release = read_parquet(f"{Configs.snomed_file_path}released_version.parquet", ['id',"term"])

# get our snomed data (for semantic tag / top category), and limit to only findings
df_concept_snomed = read_parquet(f"{Configs.snomed_file_path}concept_snomed_hug.parquet", ['id',"label","concept_type","top_category"])
df_concept_snomed = df_concept_snomed.filter((pl.col("concept_type") =="SCT_PRE") & (pl.col("top_category") == "finding"))
fiding_snomed = df_concept_snomed['id'].to_list()

# hpo graph get all nodes and useful info into dataframe

In [4]:
# Prepare rows for the node dataframe
node_rows = []

for node_id, data in graph.nodes(data=True):
    node_rows.append({
        "hpo_id": node_id,
        "label" : data.get("name", None),
        "def": data.get("def", None),
        "comment": data.get("comment", None),
        "synonyms": data.get("synonym", []),
        "xrefs": data.get("xref", []),
        "is_a": data.get("is_a", []),
        "is_obsolete": data.get("is_obsolete", "false")
    })
df_concept_hpo = pl.DataFrame(node_rows).filter(pl.col("is_obsolete") == "false")


In [5]:
# get hpo mappable list
concept_hpo_mappable = df_concept_hpo.explode("xrefs").filter(pl.col("xrefs").str.contains("SNOMED"))["hpo_id"].unique()

# get hpo non mappable dataframe
df_concept_hpo_not_mappable = df_concept_hpo.filter(~pl.col("hpo_id").is_in(concept_hpo_mappable))

# map the HPO to SNOMED

## Mappable part

In [6]:
df_concept_hpo_mappable = df_concept_hpo.filter(pl.col("hpo_id").is_in(concept_hpo_mappable))
df_concept_hpo_mappable = (df_concept_hpo_mappable
                           .select("hpo_id","label", "xrefs")
                           .explode("xrefs")
                           .filter(pl.col("xrefs").str.contains("SNOMEDCT"))
                           .with_columns(pl.col("xrefs").str.split(":"))
                           .explode("xrefs")
                           .filter(~pl.col("xrefs").str.contains("SNOMEDCT")))
df_snomed_hpo_mappable = df_concept_hpo_mappable.join(df_snomed_release, left_on="xrefs", right_on="id").rename({"label": "hpo_label", "xrefs": "snomed_id", "term": "snomed_label"})


## Non-mappable part

### finetuned lora on SapBERT embedding method

In [7]:
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn


def top_k_array_by_batch(query_matrix, candidate_matrix, batch_size=100, k=2, device="cuda"):

    # Convert to torch tensors if they are numpy arrays
    if not isinstance(query_matrix, torch.Tensor):
        query_matrix = torch.tensor(query_matrix, dtype=torch.float32, device=device)

    if not isinstance(candidate_matrix, torch.Tensor):
        candidate_matrix = torch.tensor(candidate_matrix, dtype=torch.float32, device=device)

    # Normalize vectors
    query_matrix = nn.functional.normalize(query_matrix, p=2, dim=1)
    candidate_matrix = nn.functional.normalize(candidate_matrix, p=2, dim=1)

    candidate_matrix_T = candidate_matrix.T

    num_concepts = query_matrix.shape[0]
    num_batches = (num_concepts + batch_size - 1) // batch_size

    all_top_scores = []
    all_top_indices = []

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, num_concepts)

        print(f"Processing batch {batch_idx + 1}/{num_batches} ({start_idx}-{end_idx})")

        query_batch = query_matrix[start_idx:end_idx]

        # Compute cosine similarity scores
        scores = torch.matmul(query_batch, candidate_matrix_T)

        # Directly get top-k scores and indices
        top_scores, top_indices = torch.topk(scores, k=k, dim=1)

        all_top_scores.append(top_scores.cpu())
        all_top_indices.append(top_indices.cpu())

        # Free memory explicitly if using GPU
        del scores, top_scores, top_indices
        torch.cuda.empty_cache()

    # Concatenate results from all batches
    all_top_scores = torch.cat(all_top_scores, dim=0)
    all_top_indices = torch.cat(all_top_indices, dim=0)

    return all_top_scores, all_top_indices

### embed them, save all files

In [8]:
# Download from the 🤗 Hub
base = Configs.embedding_model_hf
# mine = "yyzheng00/sapbert_lora_triplet_rank16_merged"

model = SentenceTransformer("cambridgeltl/SapBERT-from-PubMedBERT-fulltext", device=Configs.device)


No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubMedBERT-fulltext. Creating a new one with mean pooling.


In [9]:
df_snomed_release_finding = df_snomed_release.filter(pl.col("id").is_in(fiding_snomed))

df_snomed_release_finding.insert_column(0,pl.Series(np.arange(len(df_snomed_release_finding))).alias("index_snomed"))
labels_snomed = df_snomed_release_finding['term'].to_list()
index2snomedid = dict(zip(df_snomed_release_finding["index_snomed"], df_snomed_release_finding["id"]))

In [10]:
df_concept_hpo_not_mappable.insert_column(0,pl.Series(np.arange(len(df_concept_hpo_not_mappable))).alias("index_hpo"))
labels_hpo = df_concept_hpo_not_mappable['label'].to_list()
index2hpoid = dict(zip(df_concept_hpo_not_mappable["index_hpo"], df_concept_hpo_not_mappable["hpo_id"]))

In [11]:
embedding_snomed = model.encode(labels_snomed, show_progress_bar=True)
embedding_hpo = model.encode(labels_hpo,  show_progress_bar=True)



Batches:   0%|          | 0/11623 [00:00<?, ?it/s]

Batches:   0%|          | 0/492 [00:00<?, ?it/s]

In [12]:
import os
import pickle

df_snomed_release_finding.write_parquet(
    os.path.join(Configs.output_embedding_folder, "df_snomed_release_finding.parquet")
)

df_concept_hpo_not_mappable.write_parquet(
    os.path.join(Configs.output_embedding_folder, "df_concept_hpo_not_mappable.parquet")
)

np.save(
    os.path.join(Configs.output_embedding_folder, "embedding_snomed.npy"),
    embedding_snomed
)

np.save(
    os.path.join(Configs.output_embedding_folder, "embedding_hpo.npy"),
    embedding_hpo
)

with open(os.path.join(Configs.output_embedding_folder, "index2hpoid.pkl"), "wb") as f:
    pickle.dump(index2hpoid, f)

with open(os.path.join(Configs.output_embedding_folder, "index2snomedid.pkl"), "wb") as f:
    pickle.dump(index2snomedid, f)

### load all files

In [13]:
import os
import pickle



df_snomed_release_finding = pl.read_parquet(
    os.path.join(Configs.output_embedding_folder, "df_snomed_release_finding.parquet")
)

df_concept_hpo_not_mappable = pl.read_parquet(
    os.path.join(Configs.output_embedding_folder, "df_concept_hpo_not_mappable.parquet")
)

embedding_snomed = np.load(
    os.path.join(Configs.output_embedding_folder, "embedding_snomed.npy")
)

embedding_hpo = np.load(
    os.path.join(Configs.output_embedding_folder, "embedding_hpo.npy")
)

with open(os.path.join(Configs.output_embedding_folder, "index2hpoid.pkl"), "rb") as f:
    index2hpoid = pickle.load(f)

with open(os.path.join(Configs.output_embedding_folder, "index2snomedid.pkl"), "rb") as f:
    index2snomedid = pickle.load(f)

In [14]:
# for loop over hpo to find the most slimilar snomed
all_top_scores, all_top_indices = top_k_array_by_batch(embedding_hpo, embedding_snomed, batch_size=1000)

Processing batch 1/16 (0-1000)
Processing batch 2/16 (1000-2000)
Processing batch 3/16 (2000-3000)
Processing batch 4/16 (3000-4000)
Processing batch 5/16 (4000-5000)
Processing batch 6/16 (5000-6000)
Processing batch 7/16 (6000-7000)
Processing batch 8/16 (7000-8000)
Processing batch 9/16 (8000-9000)
Processing batch 10/16 (9000-10000)
Processing batch 11/16 (10000-11000)
Processing batch 12/16 (11000-12000)
Processing batch 13/16 (12000-13000)
Processing batch 14/16 (13000-14000)
Processing batch 15/16 (14000-15000)
Processing batch 16/16 (15000-15719)


In [15]:
mapped_snomed_id = []
hpo_id_to_map = []
mapped_snomed_label = []
hpo_label_to_map = []
scores_mapping = []

for hpo_index in np.arange(len(all_top_scores)):
    scores = all_top_scores[hpo_index,:]
    mapped_snomed_index = all_top_indices[hpo_index,:]
    score = scores[0]
    
    hpo_ind = index2hpoid.get(hpo_index)
    snomed_ind = index2snomedid.get(mapped_snomed_index[0].item())
    label_hpo = df_concept_hpo_not_mappable.filter(pl.col("hpo_id") == hpo_ind)['label']
    label_snomed = df_snomed_release_finding.filter(pl.col("id") == snomed_ind)['term']
    
    mapped_snomed_id.append(snomed_ind)
    hpo_id_to_map.append(hpo_ind)
    mapped_snomed_label.append(label_snomed)
    hpo_label_to_map.append(label_hpo)
    scores_mapping.append(score)

df_snomed_hpo_home_mapped = pl.DataFrame({
    "hpo_id": hpo_id_to_map,
    "hpo_label": hpo_label_to_map,
    "snomed_id": mapped_snomed_id,
    "snomed_label": mapped_snomed_label,
    "score_top_1": scores_mapping,
    "mapping_type": "sapbert"
})



In [21]:
df_hop_snomed_mapping = pl.concat([df_snomed_hpo_home_mapped,
                                df_snomed_hpo_mappable.with_columns(score_top_1 = 1.,
                                    hpo_label = pl.col("hpo_label").map_elements(lambda x: [x]),
                                    snomed_label = pl.col("snomed_label").map_elements(lambda x: [x]),
                                    mapping_type = pl.lit("official"))]
                                    ,how= "vertical")

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


In [18]:
df_hop_snomed_mapping.write_parquet(os.path.join(Configs.output_mapping_folder, "all_pheno_hpo_snomed_mapping.parquet"))

# embed diseases and get mapping to SNOMED

In [49]:
df_hop_snomed_mapping = pl.read_parquet(os.path.join(Configs.output_mapping_folder, "all_pheno_hpo_snomed_mapping.parquet"))

### embed disease

In [50]:
df_disease = df_hpoa["database_id",'disease_name'].unique()
df_disease.insert_column(0,pl.Series(np.arange(len(df_disease))).alias("index_disease"))
labels_disease = df_disease['disease_name'].to_list()
index2disease = dict(zip(df_disease["index_disease"], df_disease["database_id"]))


In [51]:
# Download from the 🤗 Hub
base = Configs.embedding_model_hf
# mine = "yyzheng00/sapbert_lora_triplet_rank16_merged"

model = SentenceTransformer("cambridgeltl/SapBERT-from-PubMedBERT-fulltext", device=Configs.device)
embedding_disease = model.encode(labels_disease, show_progress_bar=True)


No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubMedBERT-fulltext. Creating a new one with mean pooling.


Batches:   0%|          | 0/400 [00:00<?, ?it/s]

In [52]:
df_disease.write_parquet(
    os.path.join(Configs.output_embedding_folder, "df_disease.parquet")
)


np.save(
    os.path.join(Configs.output_embedding_folder, "embedding_disease.npy"),
    embedding_disease
)



with open(os.path.join(Configs.output_embedding_folder, "index2disease.pkl"), "wb") as f:
    pickle.dump(index2disease, f)


### load diseases and snomed embedding files

In [53]:
embedding_snomed = np.load(
    os.path.join(Configs.output_embedding_folder, "embedding_snomed.npy")
)
embedding_disease = np.load(
    os.path.join(Configs.output_embedding_folder, "embedding_disease.npy")
)


df_snomed_release_finding = pl.read_parquet(
    os.path.join(Configs.output_embedding_folder, "df_snomed_release_finding.parquet")
)
df_disease = pl.read_parquet(
    os.path.join(Configs.output_embedding_folder, "df_disease.parquet")
)

with open(os.path.join(Configs.output_embedding_folder, "index2snomedid.pkl"), "rb") as f:
    index2snomedid = pickle.load(f)
    
with open(os.path.join(Configs.output_embedding_folder, "index2disease.pkl"), "rb") as f:
    index2disease = pickle.load(f)

### compute similarity disease concept mappings

In [54]:
all_top_scores, all_top_indices = top_k_array_by_batch(embedding_disease, embedding_snomed, batch_size=1000)

Processing batch 1/13 (0-1000)
Processing batch 2/13 (1000-2000)
Processing batch 3/13 (2000-3000)
Processing batch 4/13 (3000-4000)
Processing batch 5/13 (4000-5000)
Processing batch 6/13 (5000-6000)
Processing batch 7/13 (6000-7000)
Processing batch 8/13 (7000-8000)
Processing batch 9/13 (8000-9000)
Processing batch 10/13 (9000-10000)
Processing batch 11/13 (10000-11000)
Processing batch 12/13 (11000-12000)
Processing batch 13/13 (12000-12770)


In [58]:
mapped_snomed_id = []
disease_id_to_map = []
mapped_snomed_label = []
disease_label_to_map = []
scores_mapping = []

for disease_index in np.arange(len(all_top_scores)):
    scores = all_top_scores[disease_index,:]
    mapped_snomed_index = all_top_indices[disease_index,:]
    score = scores[0]
    
    disease_ind = index2disease.get(disease_index)
    snomed_ind = index2snomedid.get(mapped_snomed_index[0].item())

    label_disease = df_disease.filter(pl.col("database_id") == disease_ind)['disease_name']
    label_snomed = df_snomed_release_finding.filter(pl.col("id") == snomed_ind)['term']
    
    mapped_snomed_id.append(snomed_ind)
    disease_id_to_map.append(disease_ind)
    mapped_snomed_label.append(label_snomed)
    disease_label_to_map.append(label_disease)
    scores_mapping.append(score)

df_snomed_disease_home_mapped = pl.DataFrame({
    "disease_id": disease_id_to_map,
    "disease_label": disease_label_to_map,
    "snomed_id": mapped_snomed_id,
    "snomed_label": mapped_snomed_label,
    "score_top_1": scores_mapping,
    "mapping_type": "sapbert"
})



In [60]:
df_snomed_disease_home_mapped.write_parquet(os.path.join(Configs.output_mapping_folder, "all_disease_hpo_snomed_mapping.parquet"))

# pheno - gene - disease association

## load all mapping files

In [72]:
threshold_mapping = 0.8

In [78]:
disease_mapping = pl.read_parquet(os.path.join(Configs.output_mapping_folder, "all_disease_hpo_snomed_mapping.parquet")).filter(pl.col("score_top_1")>=threshold_mapping)
pheno_mapping = pl.read_parquet(os.path.join(Configs.output_mapping_folder, "all_pheno_hpo_snomed_mapping.parquet")).filter(pl.col("score_top_1")>=threshold_mapping)
disease_mapping.head(2)

disease_id,disease_label,snomed_id,snomed_label,score_top_1,mapping_type
str,list[str],str,list[str],f64,str
"""OMIM:237800""","[""Hyperbilirubinemia, shunt, primary""]","""51071000""","[""Israel's shunt hyperbilirubinemia"", ""MAHA - Microangiopathic haemolytic anaemia"", … ""Israel's shunt hyperbilirubinaemia""]",0.911101,"""sapbert"""
"""ORPHA:83617""","[""Agammaglobulinemia-microcephaly-craniosynostosis-severe dermatitis syndrome""]","""722281001""","[""Agammaglobulinaemia, microcephaly, craniosynostosis, severe dermatitis syndrome"", ""Agammaglobulinemia, microcephaly, craniosynostosis, severe dermatitis syndrome (disorder)"", ""Agammaglobulinemia, microcephaly, craniosynostosis, severe dermatitis syndrome""]",0.989954,"""sapbert"""


In [87]:
(df_p2g
 .join(pheno_mapping, on="hpo_id")
 .select("snomed_id", "hpo_name", "gene_symbol","disease_id")
 .rename({"snomed_id" : "pheno_snomed_id"})
 .join(disease_mapping, on = "disease_id")
 .select("pheno_snomed_id", "hpo_name", "gene_symbol","snomed_id", "disease_label")
 .rename({"snomed_id" : "disease_snomed_id"})).unique().write_parquet(os.path.join(Configs.output_mapping_folder, "pheno_to_gene_snomed_mapping.parquet"))

# df_g2p

In [86]:
(df_g2p
 .join(pheno_mapping, on="hpo_id")
 .select("snomed_id", "hpo_name", "gene_symbol","disease_id")
 .rename({"snomed_id" : "pheno_snomed_id"})
 .join(disease_mapping, on = "disease_id")
 .select("pheno_snomed_id", "hpo_name", "gene_symbol","snomed_id", "disease_label")
 .rename({"snomed_id" : "disease_snomed_id"})).unique().write_parquet(os.path.join(Configs.output_mapping_folder, "gene_to_pheno_snomed_mapping.parquet"))
